In [3]:
import pandas as pd

from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/My Drive/GenAI & LLM

Mounted at /gdrive
/gdrive/My Drive/GenAI & LLM


# **Part 1: RAG Model for QA Bot**

**1. Setting Up the Environment**

In [4]:
pip install pinecone-client cohere transformers faiss-cpu torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


Import ALL Liabrary

In [5]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 52.7 MB/s eta 0:00:00


In [6]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [7]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.1 MB/s eta 0:00:00


In [8]:
import pdfplumber
import streamlit as st
from sentence_transformers import SentenceTransformer
import pinecone
import cohere

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


**2. Data Preparation**

In [9]:
# Sample dataset - documents can be more complex, such as academic papers or articles
documents = [
    "Machine learning is a subset of artificial intelligence focused on using data and algorithms to mimic how humans learn.",
    "Deep learning is a subset of machine learning where neural networks with many layers are used to process complex data.",
    "Pinecone is a vector database for machine learning applications, which allows fast and scalable retrieval of document embeddings.",
    "Cohere provides a state-of-the-art NLP API that helps in generating, classifying, and extracting text data."
]


**3. Document Embedding Creation**

In [10]:
# Load pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the documents
document_embeddings = model.encode(documents)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**4: Storing Embeddings in Pinecone**

In [11]:
pip install pinecone-client

In [15]:
from pinecone import init

In [45]:
# Initialize Pinecone with API key and environment
import pinecone

# Initialize Pinecone
pinecone.Pinecone(api_key="YOUR_API_KEY", environment="us-west1-gcp")

# Create an index
index = pinecone.Index("qa-bot",host="localhost")





In [33]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')
document_embeddings = model.encode(documents)



In [62]:
for i, embedding in enumerate(document_embeddings):
    print(i)

0
1
2
3


**5: Querying the Model**

In [73]:
# Example query
query = "What is deep learning?"

# Convert the query into an embedding
query_embedding = model.encode([query])[0]

# Query the Pinecone index to find the most similar document
# Pass the query embedding as a list
results = index.query(vector=query_embedding.tolist(), top_k=1) # Changed line

best_match_id = results['matches'][0]['id']

# Retrieve the best matching document
best_matching_doc = documents[int(best_match_id.split("_")[1])]


**6: Generating an Answer Using Cohere API**

In [75]:
# Initialize Cohere API
co = cohere.Client('YOUR_COHERE_API_KEY')

# Generate an answer based on the retrieved document and query
response = co.generate(
    model='xlarge',
    prompt=f"Answer the following question based on this context: {best_matching_doc}\n\nQuestion: {query}",
    max_tokens=100
)

# Print the generated response
print(f"Answer: {response.generations[0].text}")


# **Part 2: Interactive QA Bot Interface**

**1: Setting Up the Streamlit Interface**

In [67]:
!pip install streamlit pdfplumber


In [76]:
# Streamlit UI
st.title("Interactive QA Bot")

# File upload for PDF documents
uploaded_file = st.file_uploader("Upload a PDF document", type="pdf")

# Text input for the user's question
query = st.text_input("Ask a question based on the document")

# Function to extract text from uploaded PDF
def extract_text_from_pdf(uploaded_file):
    reader = PdfReader(uploaded_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Placeholder function to generate answer based on the query and document text
def generate_answer_from_query(query, document_text):
    # You should implement the retrieval and generation logic here
    query_embedding = model.encode(query)
    # Use Pinecone to find the most relevant document based on the query
    result = index.query(queries=[query_embedding.tolist()], top_k=1)
    # Logic to generate the answer (this is a simplified example)
    return "Answer based on closest document"

if uploaded_file is not None and query:
    # Extract text from the uploaded PDF
    document_text = extract_text_from_pdf(uploaded_file)

    # Generate the answer using the backend logic
    answer = generate_answer_from_query(query, document_text)

    # Display the answer
    st.write(f"Answer: {answer}")


2024-09-22 04:21:48.085 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-22 04:21:48.236 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-09-22 04:21:48.244 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-22 04:21:48.248 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-22 04:21:48.251 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-22 04:21:48.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-22 04:21:48.259 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-22 04:21:48.262 Thread 'MainThread': mi

**2: Backend Integration**

In [77]:
def generate_answer_from_query(query, document_text):
    # (embed document, query Pinecone, generate response via Cohere API)
    # Use functions from Part 1 to process the query and generate an answer
    return answer

**3: Containerization Using Docker**

In [84]:
# Use the official Python image from Docker Hub
FROM python:3.9-slim

# Set the working directory in the container
WORKDIR /app
WORKDIR /app
WORKDIR /app

# Copy the requirements.txt into the container
COPY requirements.txt .

# Install the dependencies specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Copy the rest of the application code into the container
COPY . .

# Expose the port that Streamlit uses (default is 8501)
EXPOSE 8501

# Command to run the Streamlit app
CMD ["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0"]


In [87]:
!docker build -t qa-bot .
!docker run -p 8501:8501 qa-bot